# Lectura y preprocesamiento de datos

In [98]:
import pandas as pd
import datetime
import numpy as np
from IPython.display import clear_output

import tensorflow as tf

In [2]:
data = pd.read_excel("data.xlsb")
data["Fecha_Reporte"] = data["Fecha_Reporte"].apply(lambda x : pd.to_datetime("1899-12-30") + datetime.timedelta(days=x))
data["Fecha_Ocurrencia"] = data["Fecha_Ocurrencia"].apply(lambda x : pd.to_datetime("1899-12-30") + datetime.timedelta(days=x))
data["Fecha_Pago"] = data["Fecha_Pago"].apply(lambda x : pd.to_datetime("1899-12-30") + datetime.timedelta(days=x))

In [65]:
aux = data.groupby(["Fecha_Pago"])["Importe USD"].sum().reset_index()

In [66]:
fecha_inicial = aux["Fecha_Pago"].min()
fecha_final = aux["Fecha_Pago"].max()

rango_fechas = pd.date_range(fecha_inicial, fecha_final, freq="D")

In [67]:
#Si una fecha no tiene registro, se agrega con importe 0
aux = aux.set_index("Fecha_Pago")
aux = aux.reindex(rango_fechas, fill_value = 0)
aux = aux.reset_index(names = ["Fecha_Pago", "Importe USD"])

In [69]:
X = aux[["Fecha_Pago"]]
y = aux[["Importe USD"]]

In [70]:
X["anno"] = X["Fecha_Pago"].apply(lambda x : x.year)
X["Mes"] = X["Fecha_Pago"].apply(lambda x : x.month)
X["Dia"] = X["Fecha_Pago"].apply(lambda x : x.day)
X["Dia_Semana"] = X["Fecha_Pago"].apply(lambda x : x.weekday())

In [71]:
#Ahora cada uno de estos valores los transformaremos a un encoding cíclico.
X["Dia_Semana_sin"] = X["Dia_Semana"].apply(lambda x : np.sin(2*np.pi*x/7))
X["Dia_Semana_cos"] = X["Dia_Semana"].apply(lambda x : np.cos(2*np.pi*x/7))

In [72]:
X["Mes_sin"] = X["Mes"].apply(lambda x : np.sin(2*np.pi*x/12))
X["Mes_cos"] = X["Mes"].apply(lambda x : np.cos(2*np.pi*x/12))

In [73]:
#Los días dependerán del mes, pues hay meses que tienen 30 días y otros 31, además de febrero que tiene 28 o 29.
X["Dia_sin"] = X.apply(lambda x : np.sin(2*np.pi*x["Dia"]/x["Fecha_Pago"].days_in_month), axis=1)
X["Dia_cos"] = X.apply(lambda x : np.cos(2*np.pi*x["Dia"]/x["Fecha_Pago"].days_in_month), axis=1)

In [74]:
#El año lo estandarizamos, pero comenzando desde 0.
X["anno"] = (X["anno"] - X["anno"].min()) / (X["anno"].max() - X["anno"].min())

In [75]:
columns = ["anno", "Dia_Semana_sin", "Dia_Semana_cos", "Mes_sin", "Mes_cos", "Dia_sin", "Dia_cos"]

In [79]:
#Normalizamos los valores de y
y = (y.values - y.values.mean()) / y.values.std()

In [86]:
#Creamos el X_train e y_train utilizando secuencias de un largo fijo
sequence_length = 30
X_train = []
y_train = []

for i in range(sequence_length, len(X)):
    X_train.append(X.iloc[i-sequence_length:i][columns].values)
    y_train.append(y.iloc[i])

X_train = np.array(X_train)
y_train = np.array(y_train)

In [88]:
X_train.shape, y_train.shape

((5071, 30, 7), (5071, 1))

# Implementación del modelo

In [99]:
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.LSTM(128, input_shape = (sequence_length, X_train.shape[2])),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    model.compile(loss='mean_squared_error', optimizer='adam')

    return model

In [100]:
best_model = None
best_loss = float("inf")
loss = float("inf")

for i in range(20):

    print(f"BEST LOSS: {best_loss}")
    print(f"Iteration {i} - Loss: {loss}")

    model = create_model()
    model.fit(X_train, y_train, epochs = 1000, batch_size = 32)

    loss = model.evaluate(X_train, y_train, verbose=0)

    if loss < best_loss:
        best_model = model
        best_loss = loss

    clear_output(wait=True)

BEST LOSS: inf
Iteration 0 - Loss: inf
Epoch 1/1000
159/159 [==============================] - 3s 8ms/step - loss: 13441511424.0000
Epoch 2/1000
159/159 [==============================] - 1s 6ms/step - loss: 13438504960.0000
Epoch 3/1000
159/159 [==============================] - 1s 6ms/step - loss: 13433799680.0000
Epoch 4/1000
159/159 [==============================] - 1s 7ms/step - loss: 13427179520.0000
Epoch 5/1000
159/159 [==============================] - 1s 7ms/step - loss: 13418983424.0000
Epoch 6/1000
159/159 [==============================] - 1s 7ms/step - loss: 13409910784.0000
Epoch 7/1000
159/159 [==============================] - 1s 7ms/step - loss: 13401107456.0000
Epoch 8/1000
159/159 [==============================] - 1s 6ms/step - loss: 13392032768.0000
Epoch 9/1000
159/159 [==============================] - 1s 7ms/step - loss: 13383112704.0000
Epoch 10/1000
159/159 [==============================] - 1s 7ms/step - loss: 13375070208.0000
Epoch 11/1000
159/159 [=======